# Student Loan Risk with Deep Learning

In [1]:
# Imports
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [4]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

1    855
0    744
Name: credit_ranking, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [5]:
# Define the target set y using the credit_ranking column
y = np.array(loans_df['credit_ranking'])

# Display a sample of y
y[0:5]

array([0, 0, 0, 1, 0], dtype=int64)

In [6]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop(columns='credit_ranking')

# Review the features DataFrame
X.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [7]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [8]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Scale the features training dataset
X_train_scaled = X_scaler.transform(X_train)

# Scale the test training dataset
X_test_scaled = X_scaler.transform(X_test)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [9]:
# Define the the number of inputs (features) to the model
input_nodes = len(X.columns)

# Review the number of features
input_nodes

11

In [10]:
# Define the number of hidden nodes for the first hidden layer
layer1_nodes = 6

# Define the number of hidden nodes for the second hidden layer
layer2_nodes = 3

# Define the number of neurons in the output layer
output_nodes = 1

In [11]:
# Create the Sequential model instance
nn_model = tf.keras.models.Sequential()

# Add the first hidden layer
nn_model.add(tf.keras.layers.Dense(units=layer1_nodes, activation='relu', input_dim=input_nodes))

# Add the second hidden layer
nn_model.add(tf.keras.layers.Dense(units=layer2_nodes, activation='relu'))

# Add the output layer to the model specifying the number of output neurons and activation function
nn_model.add(tf.keras.layers.Dense(units=output_nodes, activation='sigmoid'))

In [12]:
# Display the Sequential model summary
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 72        
                                                                 
 dense_1 (Dense)             (None, 3)                 21        
                                                                 
 dense_2 (Dense)             (None, 1)                 4         
                                                                 
Total params: 97 (388.00 Byte)
Trainable params: 97 (388.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [13]:
# Compile the Sequential model
nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
# Fit the model using 50 epochs and the training data
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50


38/38 [==============================] - 1s 940us/step - loss: 0.7201 - accuracy: 0.4787
Epoch 2/50
38/38 [==============================] - 0s 889us/step - loss: 0.6978 - accuracy: 0.5088
Epoch 3/50
38/38 [==============================] - 0s 877us/step - loss: 0.6882 - accuracy: 0.5771
Epoch 4/50
38/38 [==============================] - 0s 856us/step - loss: 0.6826 - accuracy: 0.6105
Epoch 5/50
38/38 [==============================] - 0s 933us/step - loss: 0.6784 - accuracy: 0.6330
Epoch 6/50
38/38 [==============================] - 0s 903us/step - loss: 0.6746 - accuracy: 0.6347
Epoch 7/50
38/38 [==============================] - 0s 890us/step - loss: 0.6707 - accuracy: 0.6405
Epoch 8/50
38/38 [==============================] - 0s 909us/step - loss: 0.6666 - accuracy: 0.6564
Epoch 9/50
38/38 [==============================] - 0s 902us/step - loss: 0.6622 - accuracy: 0.6539
Epoch 10/50
38/38 [==============================] - 0s 901us/step - loss: 0.6573 - accuracy: 0.66

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [15]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_model.evaluate(X_test, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

13/13 - 0s - loss: 10.6656 - accuracy: 0.5000 - 115ms/epoch - 9ms/step
Loss: 10.665627479553223, Accuracy: 0.5


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [16]:
# Set the model's file path
file_path = Path('student_loans.keras')

# Export your model to a keras file
nn_model.save(file_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [17]:
# Set the model's file path
file_path = Path('student_loans.keras')

# Load the model to a new object
nn_imported = tf.keras.models.load_model(file_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [18]:
# Make predictions with the test data
predictions = nn_imported.predict(X_test_scaled, verbose=2)

# Display a sample of the predictions
predictions[0:5]

13/13 - 0s - 67ms/epoch - 5ms/step


array([[0.6192903 ],
       [0.48057598],
       [0.7407941 ],
       [0.7129672 ],
       [0.7516452 ]], dtype=float32)

In [19]:
# Save the predictions to a DataFrame and round the predictions to binary results
predictions_df = pd.DataFrame(predictions, columns=['predictions'])
predictions_df["predictions"] = round(predictions_df["predictions"],0)

### Step 4: Display a classification report with the y test data and predictions

In [20]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, predictions_df['predictions'].values))

              precision    recall  f1-score   support

           0       0.73      0.74      0.73       188
           1       0.77      0.75      0.76       212

    accuracy                           0.75       400
   macro avg       0.75      0.75      0.75       400
weighted avg       0.75      0.75      0.75       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**


**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**


**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**



### Discussion Answers:

1. To build a recommendation system to recommend student loan options for students, I would need to collect personal information such as age, location and citizenship status to determine loan availability and eligibility since different countries offer different types of financing for student loans. I would also need to gather finicancial information such as income level, employment status, credit score, current assets and current debts. These factors would help to determine the financial need of the student and also affect their loan eligibility, particularly when applying for a private student loan. Academic data such as which schools the student is applying to, what degree they are seeking and what field of study they are entering, as well as their academic record could also affect their ability to qualify or pay back their loan in the future. Finally, loan product information and eligibility requirements would be necessary so that students would not get recommended to apply for a loan that they do not qualify for. 

2. Based on the data I chose to use, my model would use a combination of content-based filtering and context-based filtering. Content based filtering would be used to match loan products to students financial needs, ability to repay the loan, and eligibility for the loan. Context-based filtering would be used in a similar manner to determine the student's loan repayment ability since context based features such as current interest rates and projected income based on which school and field of study the student is entering are constantly changing. Since picking a student loan is determined largely on picking the loan that meets the students financial needs with the lowest interest rate, it would not make since to use collaborative filtering in this context.

3. Two real-world challenges that I would take into consideration while building a recommendation system for student loans are keeping up to date with student loan products and streamlining the loan application process, possibly by combining it with the recommendation system. Since available loan products are constantly changing, it would be vital to have current information in this area to ensure that students are seeing the best possible loans on the market and that those loans are currently available. Since most students fill out the FAFSA application to apply for federal student loans, it would be convinient if the recommendation system could read in a FAFSA form. Since the information collected for the recommendation system is very similar to the information used to apply for a loan, it would also be convinient if the recommendation system could fill out loan applications with the information that was provided to it.  